# Valores Faltantes

Se definen como valores / datos no disponibles para ciertas observaciones en una variable que serían útiles para el análisis de los resultados.

- Aprenderemos como detectar y cuantificar valores faltantes Usaremos los datos de Titanic y loan


In [1]:
#Cargamos librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Cargamos los datos del Titanic
data = pd.read_csv('../titanic.csv')

# Inspeccionemos las primeras 5 filas
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
# Podemos cuantificar el número total de valores nulos usando el método isnull 

data.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

Podemos observar que hay 263 valores null para la variable **age**, 1014 para la variable **cabin**, 2 par la variable **embarked**

In [5]:
# También podemos usar el método 'mean' para visualizar el porcentage de los valores faltantes  por cada variable

data.isnull().mean()*100

pclass        0.000000
survived      0.000000
name          0.000000
sex           0.000000
age          20.091673
sibsp         0.000000
parch         0.000000
ticket        0.000000
fare          0.076394
cabin        77.463713
embarked      0.152788
boat         62.872422
body         90.756303
home.dest    43.086325
dtype: float64

 Age (20% faltantes), Cabin (77% faltantes),  Embarked  (0.2%  faltantes).
 

## Mecanismos de datos Faltantes

### Valores Faltantes No Aleatorios, MNAR ('Missing Data not at Random')

Los valores faltantes no son aleatorios (MNAR) cuando hay un mecanismo o una razón por la cual los valores faltantes son introducidos en los dato

En los datos del Titanic, los valores faltantes de las variables Cabin y Age fueron introducidos sistemáticamente. Para muchos de los pasajeros que no sobrevivieron, la edad ('Age') que tenían o la cabina ('Cabin') en la que estaban viajando, no pudo ser establecida. A la gente que sobrevivió, por el contrario, se les pudo preguntar dicha información.

Evaluemos el porcentaje de los valores faltantes en la cabina por los pasajeros que sobrevivieron vs. los que no sobrevivieron
La variable **'Survived'** toma valores de 1 si el pasajero sobrevivió y 0 de lo contrario.
Agruparermos los datos para Sobrevivientes vs No-Sobrevivientes

In [6]:
data['cabin'].isnull().groupby(data['survived']).mean()

survived
0    0.873918
1    0.614000
Name: cabin, dtype: float64

Podemos ver que el porcentaje de valores faltantes es mucho más alto para las personas que no sobrevivieron (87%), con respecto a los que sobrevivieron (60%). Este hallazgo está alineado con nuestra hipótesis sobre los datos faltantes.

**Nota:** Para realmente entender si los valores faltantes no son aleatorios, es necesario familiarizarse muy bien con la forma en que los datos fueron recogidos. Analizar los datos, solo puede llevarnos a la dirección correcta o ayudarnos a construir hipótesis.

**Revisemos la variable Age**

In [7]:
data['age'].isnull().groupby(data['survived']).mean()

survived
0    0.234858
1    0.146000
Name: age, dtype: float64

Nuevamente podemos observar un porcentaje mayor de datos faltantes para los pasajeros que no sobrevivieron. Este análisis sugiere que hay una pérdida de datos sistemática: la gente que no sobrevivió tiende a tener más datos nulos. 
Probablemente, el método seleccionado para recoger la información, contribuye a la generación de los datos faltantes.

### Datos Faltantes Completamente Aleatorios- MCAR (Missing Data Completely at Random)

Cuando los datos son de tipo MCAR significa que la falta de datos no está relacionada con ningún otro valor observado o no observado. En otras palabras, la probabilidad de que un valor falte es igual para todos los casos, independientemente de cualquier otra característica o valor en los datos. 

Tomemos un segmento del dataframe con solo las observaciones con datos faltantes para la variable Embarked

In [10]:

data[data.embarked.isnull()]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
168,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,6,NaN,NaN
284,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,6,NaN,"Cincinatti, OH"


Estas dos mujeres viajaban juntas, Miss Icard era la mucama de Mrs Stone. No hay indicios de que los valores nulos de la variable **"embarked"** dependan de ninguna otra variable. Además, dado que ambas mujeres sobrevivieron al naufragio, se podría suponer que podrían haber proporcionado esta información. Sin embargo, es probable que los valores se hayan perdido al momento de crear los datos. En otras palabras, si los valores son MCAR, la probabilidad de que faltaran los datos para estas dos mujeres es la misma que para cualquier otra persona en el Titanic. Aunque esta hipótesis puede ser difícil de comprobar, al menos nos sirve como un ejemplo de variables MCAR.

### Valores Faltantes Aleatorios MAR ('Missing Data at Random')

Ocurre cuando hay una relación entre la probabilidad de ocurrencia de los datos faltantes y los valores existentes. En otras palabras, la probabilidad de que una observación falte depende de la disponibilidad de la información (es decir, de otras variables en los datos).

Para este ejemplo, usaremos los datos loan. Vamos a explorar la variable ocupacion (employment) y los años que lleva empleado (time_employed), ambos declarados por los prestamistas en el momento de aplicación para un préstamo.

En este ejemplo, los valores faltantes en employment estan asociados con los valores faltantes en time_employed.

In [11]:
data = pd.read_csv('../loan.csv', usecols=['employment', 'time_employed'])

data.head()

,employment,time_employed
0,Teacher,<=5 years
1,Accountant,<=5 years
2,Statistician,<=5 years
3,Other,<=5 years
4,Bus driver,>5 years


In [12]:
# Revisamos el porcentaje de datos faltantes

data.isnull().mean()

employment       0.0611
time_employed    0.0529
dtype: float64

Aldedor del 6%  contienen datos faltantes para ambras variables.

In [21]:
# Miremos los valores de los diferentes empleos

# Número de los diferentes empleos:
print(f"'Número de los diferentes empleos: {len(data['employment'].unique())}")

# Veamos unos ejemplos:
data['employment'].unique()

'Número de los diferentes empleos: 12


array(['Teacher', 'Accountant', 'Statistician', 'Other', 'Bus driver',
       'Secretary', 'Software developer', 'Nurse', 'Taxi driver', nan,
       'Civil Servant', 'Dentist'], dtype=object)

In [22]:
# Veamos la variable time_employed

data['time_employed'].unique()

array(['<=5 years', '>5 years', nan], dtype=object)

El cliente no puede introducir un valor para el tiempo de empleo si no está empleado. Pueden ser estudiantes, jubilados, trabajadores autónomos o trabajar en la casa. Pero vemos cómo estas 2 variables están relacionadas entre sí.

In [25]:
# Calculemos la proporción de datos que faltan para la variable time_employed
# para los clientes que declararon el empleo

# filtramos en donde empleado no es null
t = data[~data['employment'].isnull()]

# porcentaje de datos faltantes en el tiempo empleado
t['time_employed'].isnull().mean()


0.0005325380764724678

In [27]:
# Hagamos lo mismo para los que no lo hicieron... 


# clientes que no declararon el empleo null
t = data[data['employment'].isnull()]

# porcentaje de datos faltantes en el tiempo empleado
t['time_employed'].isnull().mean()

0.8576104746317512

El número de registros que han informado de la ocupación y tienen valores perdidos en time_employed es mínimo. Mientras que los clientes que no reportaron una ocupación o empleo, están reportando mayormente valores faltantes en la variable time_employed

Esto apoya aún más que los valores que faltan en el empleo están relacionados con los valores que faltan en time_employed.